In [ ]:
from __future__ import annotations

import sys
import time

import mlflow
import onnx
import torch as th
from pettingzoo.test import api_test
from sb3_contrib import MaskablePPO
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from stable_baselines3.common.logger import HumanOutputFormat, Logger

from lib.action_mask_wrapper import SB3ActionMaskWrapper
from lib.briscola.game import BriscolaGame
from lib.briscola_env.briscola_env import BriscolaEnv
from lib.briscola_env.embedding import game_embedding
from lib.mlflow_logging import MLflowOutputFormat
from lib.onnxable import OnnxableMaskableACPolicy

In [ ]:
PLAYER_COUNT = 4
NUM_TRAINING_STEPS = 150_000
NUM_EVAL_GAMES = 500

MODEL_HYPERPARAMS = {
	"n_steps": 40 * 4,
	"batch_size": 40,
	"learning_rate": 3e-4,
	"ent_coef": 0.01,
	"vf_coef": 0.65,
	"normalize_advantage": True,
}

def make_env():
	return BriscolaEnv(num_players=PLAYER_COUNT)

def make_model(env):
	return MaskablePPO(
        MaskableActorCriticPolicy, 
        env, 
        verbose=1,
        **MODEL_HYPERPARAMS,
    )

api_test(make_env(), num_cycles=1000)


In [ ]:
# Training function
def train(
    steps: int = 10_000, seed: int | None = 0, **env_kwargs
):
    # Train a single model to play as each agent in a cooperative Parallel environment
    env = SB3ActionMaskWrapper(make_env())
    env.reset(seed=seed)

    def mask_fn(env):
        return env.action_mask()
    env = ActionMasker(env, mask_fn)

    loggers = [HumanOutputFormat(sys.stdout), MLflowOutputFormat()]

    print(f"Starting training on {str(env.metadata)}.")
    model = make_model(env)
    model.set_random_seed(seed)
    model.set_logger(Logger(folder=None, output_formats=loggers))
    model.learn(total_timesteps=steps)
    model_path = f"{env.unwrapped.metadata.get('name')}_{time.strftime('%Y%m%d-%H%M%S')}"
    model.save(model_path)

    print("Model has been saved.")
    print(f"Finished training on {str(env.unwrapped.metadata['name'])}.\n")
    env.close()
    return model, model_path


In [ ]:
# Evaluation Code
def make_results(agents, wins, total_rewards):
    player_results = []
    total_wins = sum(wins.values())
    print("Winrates:")
    for p in agents:
        winrate = wins[p] / total_wins
        print(f"\t{p}: {winrate*100}%")
        print(f"\tWins: {wins[p]} Rewards: {total_rewards[p]}")
        player_results.append({"winrate": winrate, "total_rewards": total_rewards[p]})
    return player_results

def eval_action_mask(player, model_path, num_games=100):
    # Evaluate a trained agent vs a random agent
    env = make_env()
    model = None
    print("Starting evaluation vs random agents.")
    if model_path is not None:
        print(f"Trained agent will play as {env.possible_agents[player]}")
        print("Using policy:", model_path)
        model = MaskablePPO.load(model_path)

    wins = {agent: 0 for agent in env.possible_agents}
    total_rewards = {agent: 0 for agent in env.possible_agents}
    for i in range(num_games):
        env.reset(seed=i)
        for agent in env.agent_iter():
            obs, reward, termination, truncation, info = env.last()
            observation, action_mask = obs.values()
            if termination or truncation:
                winner = max(env.rewards, key=env.rewards.get)
                wins[winner] += env.infos[winner]["wins"] 
                for a in env.possible_agents:
                    total_rewards[a] += env.rewards[a]
                break
            else:
                # Act randomly unless it's the agents turn
                if model is None or agent != env.possible_agents[player]:
                    act = env.action_space(agent).sample(action_mask)
                else:
                    act = int(
                        model.predict(
                            observation, action_masks=action_mask, deterministic=True
                        )[0]
                    )
            env.step(act)
    env.close()
    return make_results(env.agents, wins, total_rewards)


In [ ]:
# Plotting Code
import matplotlib.pyplot as plt
import numpy as np

def make_plot(winrate_data, title):
	winrates = np.transpose(np.array([
		[entry['winrate'] for entry in sublist] 
		for sublist in winrate_data
   	]))
	fig, ax = plt.subplots(figsize=(8, 6))

	# How many groups and bars
	num_positions, num_players = winrates.shape
	x = np.arange(num_players)

	# Width of each bar group
	bar_width = 0.2
	
	# Plot each group's bars
	for i in range(num_positions):
		ax.bar(x + i * bar_width, winrates[i], width=bar_width, label=f'Player {i}')

	# Labels and stuff
	ax.set_xlabel('Model Position')
	ax.set_ylabel('Winrate')
	ax.set_title(title)
	ax.set_xticks(x + bar_width * (num_positions-1) / 2)
	ax.set_xticklabels([str(i) for i in range(num_players)])
	ax.legend()

	plt.tight_layout()
	plt.show()
	plt.close(fig)
	return fig


In [ ]:
# Export to Onnx
def onnx_export(model_path):
    model_source_file = model_path
    model_dest_file = f"{model_path}-probdist.onnx"
    model = MaskablePPO.load(model_source_file, device="cpu")
    onnx_policy = OnnxableMaskableACPolicy(model.policy)
    observation_size = model.observation_space.shape
    dummy_input = th.randn(1, *observation_size)
    th.onnx.export(
        onnx_policy,
        dummy_input,
        model_dest_file,
        opset_version=17,
        input_names=["input"],
        output_names=["dist", "values"]
    )
    onnx_model = onnx.load(model_dest_file)
    print(f"Onnx model exported to {model_dest_file}")
    return onnx_model, model_dest_file

In [ ]:
# Train & Track

mlflow.set_tracking_uri("http://localhost:8081")
mlflow.set_experiment("Briscola AI")

with mlflow.start_run():
    mlflow.set_tag("Training Info", f"{PLAYER_COUNT} player model")
    mlflow.log_param("num_players", PLAYER_COUNT)
    mlflow.log_param("num_training_steps", NUM_TRAINING_STEPS)
    mlflow.log_param("num_eval_games", NUM_EVAL_GAMES)
    mlflow.log_params(MODEL_HYPERPARAMS)

    training_result, model_path = train(steps=NUM_TRAINING_STEPS)
    onnx_model, onnx_file = onnx_export(model_path)

    baseline_results = eval_action_mask(-1, None, num_games=NUM_EVAL_GAMES)
    results_by_position = []
    for position in range(PLAYER_COUNT):
        results_by_position.append(eval_action_mask(position, model_path, num_games=NUM_EVAL_GAMES))

    fig = make_plot([baseline_results], "All random policy")
    mlflow.log_figure(fig, "basline_random_policy.png")
    fig = make_plot(results_by_position, "Winrates by Model player position")
    mlflow.log_figure(fig, "winrates_by_position.png")

    for i, result in enumerate(baseline_results):
            mlflow.log_metric(
                f"baseline_winrate_player_{i}", baseline_results[i]["winrate"]
            )
            mlflow.log_metric(
                f"baseline_total_rewards_player_{i}", baseline_results[i]["total_rewards"]
            )

    for i, result in enumerate(results_by_position):
        for j, player in enumerate(result):
            mlflow.log_metric(f"model_pos_{i}_winrate_player_{j}", player["winrate"])
            mlflow.log_metric(f"model_pos_{i}_total_rewards_player_{j}", player["total_rewards"])

    example_input = game_embedding(BriscolaGame(players=PLAYER_COUNT), 0)
    sig = mlflow.models.infer_signature(example_input)
    mlflow.pytorch.log_model(
        training_result.policy,
        model_path,
        signature=sig,
        registered_model_name="briscola",
    )
    mlflow.onnx.log_model(
        onnx_model,
        onnx_file,
        signature=sig,
        registered_model_name="briscola_onnx",
    )
